In [20]:
# from google.colab import files
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot

from scipy import stats
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2


from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from sklearn import ensemble
from xgboost import XGBRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor


from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor

import statsmodels.api as sm

In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
df = pd.read_csv('/kaggle/input/lab-02-introduction-to-machine-learning/train.csv')
df.head()

In [23]:
df.isna().sum()

In [24]:
print(df.shape)
df.drop_duplicates(subset=df.columns.difference(['ID']),inplace=True)
print(df.shape)

In [25]:
df.describe()

In [26]:
x = df.loc[:, ['AT','V','AP','RH']]
y = df['EP']

In [27]:
def metrics(ytest,pred,trainx):
  print("MAE", mae(ytest, pred))
  print("MSE", mse(ytest,pred))
  print("RMSE", np.sqrt(mse(ytest,pred)))
  r2score = r2(ytest,pred)
  print("r2score", r2score)
  n=len(trainx)
  k=4
  adj_r2_score = 1 - ((1-r2score)*(n-1)/(n-k-1))
  print("adjr2lis", adj_r2_score)

In [ ]:
#for train, validation and test split
trainx, X_test, trainy, y_test = train_test_split(x, y, test_size=0.2, random_state = 0) #, stratify=y

In [29]:
scaler = StandardScaler()
# transform data
stdscaled = x.copy(deep=True)
stdscaled[stdscaled.columns] = scaler.fit_transform(stdscaled[stdscaled.columns])
# stdscaled = scaler.fit_transform(x)
stdscaled.head()

In [30]:
scaler = MinMaxScaler()
# transform data
minscaled = x.copy(deep=True)
minscaled[minscaled.columns] = scaler.fit_transform(minscaled[minscaled.columns])
# stdscaled = scaler.fit_transform(x)
minscaled.head()

In [31]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0,test_size=0.2)

In [32]:
models = []
models.append(('LR', LinearRegression()))
models.append(('SVR', SVR()))
models.append(('DT', DecisionTreeRegressor()))
models.append(('RF', RandomForestRegressor()))
models.append(('KN', KNeighborsRegressor()))
models.append(('XG', XGBRegressor()))
models.append(('LG',LGBMRegressor()))
models.append(('GBT',ensemble.GradientBoostingRegressor()))

In [33]:
names = []
scores = []
maes = []
# for name, model in models:
#     model.fit(xtrain, ytrain)
#     y_pred = model.predict(xtest)
#     scores.append(accuracy_score(ytest, y_pred))
#     maes.append(mae(ytest, y_pred))
#     names.append(name)
# tr_split = pd.DataFrame({'Name': names, 'Score': scores, 'MAE': maes})
# print(tr_split)
scorer = make_scorer(mae)
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True) 
    score = cross_val_score(model, x, y, cv=kfold, scoring=scorer).mean()
    
    names.append(name)
    scores.append(score)

kf_cross_val = pd.DataFrame({'Name': names, 'Score': scores})
print(kf_cross_val)

In [34]:
names = []
scores = []
maes = []

scorer = make_scorer(mae)
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True) 
    score = cross_val_score(model, stdscaled, y, cv=kfold, scoring=scorer).mean()
    
    names.append(name)
    scores.append(score)

kf_cross_val = pd.DataFrame({'Name': names, 'Score': scores})
print(kf_cross_val)

In [35]:
names = []
scores = []
maes = []

scorer = make_scorer(mae)
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True) 
    score = cross_val_score(model, minscaled, y, cv=kfold, scoring=scorer).mean()
    
    names.append(name)
    scores.append(score)

kf_cross_val = pd.DataFrame({'Name': names, 'Score': scores})
print(kf_cross_val)

In [36]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [25,50,75,100,125,150]# 300, 1000
}
rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(minscaled, y)
print("Best parameters:", grid_search.best_params_)

In [37]:
xtrain, xtest, ytrain, ytest = train_test_split(minscaled, y, random_state=0,test_size=0.2)
rf = RandomForestRegressor(random_state = 0, n_estimators = 150,max_depth=90,min_samples_leaf=3,min_samples_split=8,bootstrap=True)
rf.fit(xtrain, ytrain)
pred = rf.predict(xtest)
mae(ytest, pred)

In [38]:
xtrain, xtest, ytrain, ytest = train_test_split(minscaled, y, random_state=0,test_size=0.2)
for val in [5,10,20,50,100,200,250,500,750,1000,1250,1500,1750,2000]:
  rf = RandomForestRegressor(n_estimators = val, random_state = 0)
  rf.fit(xtrain, ytrain)
  pred = rf.predict(xtest)
  maescore = mae(ytest, pred)
  print("n_est: " + str(val) + " and MAE : " + str(maescore))

In [39]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0,test_size=0.2)
lgb = LGBMRegressor(max_depth=75,learning_rate=0.1,num_leaves=2*75)
lgb.fit(xtrain, ytrain)
pred = lgb.predict(xtest)
maescore = mae(ytest, pred)
print(maescore)

In [40]:
xtrain, xtest, ytrain, ytest = train_test_split(minscaled, y, random_state=0,test_size=0.2)
lgb = LGBMRegressor(max_depth=75,learning_rate=0.1,num_leaves=2*75)
lgb.fit(xtrain, ytrain)
pred = lgb.predict(xtest)
maescore = mae(ytest, pred)
print(maescore)

In [41]:
xtrain, xtest, ytrain, ytest = train_test_split(stdscaled, y, random_state=0,test_size=0.2)
lgb = LGBMRegressor(max_depth=75,learning_rate=0.1,num_leaves=2*75)
lgb.fit(xtrain, ytrain)
pred = lgb.predict(xtest)
maescore = mae(ytest, pred)
print(maescore)

In [42]:
xtrain, xtest, ytrain, ytest = train_test_split(minscaled, y, random_state=0,test_size=0.2)
for val in [1,5,10,15,20,25,30,50,75,100]:
  for l in [0.1]: #0.0001, 0.001, 0.01, 0.1, 1.0
    lgb = LGBMRegressor(max_depth=val,learning_rate=l,num_leaves=2*val)
    lgb.fit(xtrain, ytrain)
    pred = lgb.predict(xtest)
    maescore = mae(ytest, pred)
    print("max_depth: " + str(val) + " and rate : " + str(l) + " and MAE : " + str(maescore))
    print("================================")

In [43]:
xtrain, xtest, ytrain, ytest = train_test_split(stdscaled, y, random_state=0,test_size=0.2)
for val in [1,5,10,15,20,25,30,50,75,100]:
  for l in [0.1]: #0.0001, 0.001, 0.01, 0.1, 1.0
    lgb = LGBMRegressor(max_depth=val,learning_rate=l,num_leaves=2*val)
    lgb.fit(xtrain, ytrain)
    pred = lgb.predict(xtest)
    maescore = mae(ytest, pred)
    print("max_depth: " + str(val) + " and rate : " + str(l) + " and MAE : " + str(maescore))
    print("================================")

In [44]:
maxdepth=[]
# learningrate = []
maelis=[]
mselis = []
rmselis=[]
r2lis=[]
adjr2lis=[]

In [45]:
x.head()

In [46]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0,test_size=0.2)
for val in [5,10,15,25,50,75,100,125,150]:
  lgb = LGBMRegressor(max_depth=val,learning_rate=0.1,num_leaves=2*val)
  lgb.fit(xtrain, ytrain)
  pred = lgb.predict(xtest)
  maxdepth.append(val)
  # learningrate.append(l)
  maelis.append(mae(ytest, pred))
  mselis.append(mse(ytest,pred))
  rmselis.append(np.sqrt(mse(ytest,pred)))
  r2score = r2(ytest,pred)
  r2lis.append(r2score)
  n=len(x)
  k=4
  adj_r2_score = 1 - ((1-r2score)*(n-1)/(n-k-1))
  adjr2lis.append(adj_r2_score)
  # print("max_depth: " + str(val) + " and rate : " + str(l) + " and MAE : " + str(maescore))
  # print("================================")

In [47]:
lgbdf = pd.DataFrame(list(zip(maxdepth,maelis,mselis,rmselis,r2lis,adjr2lis)), columns =['max_depth', 'MAE','MSE','RMSE','RSquared','Adjusted RSquared'])
lgbdf

In [48]:
nest=[]
gamlis = []
maxlis = []
maelis=[]
mselis = []
rmselis=[]
r2lis=[]
adjr2lis=[]

In [49]:
#XGBRegressor(n_estimators=1000,gamma=1,max_depth=7,objective='reg:squarederror')
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0,test_size=0.2)
for val in [500,1000]:
  for l in [1,3]: 
    for m in [7,10]:
      model = XGBRegressor(n_estimators=val,gamma=l,max_depth=m,objective='reg:squarederror')
      model.fit(xtrain, ytrain)
      pred = model.predict(xtest)
      nest.append(val)
      gamlis.append(l)
      maxlis.append(m)
      maelis.append(mae(ytest, pred))
      mselis.append(mse(ytest,pred))
      rmselis.append(np.sqrt(mse(ytest,pred)))
      r2score = r2(ytest,pred)
      r2lis.append(r2score)
      n=len(x)
      k=4
      adj_r2_score = 1 - ((1-r2score)*(n-1)/(n-k-1))
      adjr2lis.append(adj_r2_score)

In [50]:
xgbdf = pd.DataFrame(list(zip(nest,gamlis,maxlis,maelis,mselis,rmselis,r2lis,adjr2lis)), columns =['n_estimators','gamma','max_depth', 'MAE','MSE','RMSE','RSquared','Adjusted RSquared'])
xgbdf

In [51]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0,test_size=0.2)
model = XGBRegressor(n_estimators=1000,gamma=1,max_depth=7,objective='reg:squarederror')
model.fit(xtrain, ytrain)
print("Training Accuracy: - ")
metrics(ytrain,model.predict(xtrain),xtrain)
print("========================================================")
pred = model.predict(xtest)
metrics(ytest,pred,xtest)

In [52]:
def plotscores(df,parameter,scorecols,model):
  fig = plt.figure(figsize=(8, 6))
  colors = ['blue','orange','green']
  i=0
  for col in scorecols: 
    plt.plot(df[parameter], df[col], label=col, 
            linewidth=3, marker='o',color=colors[i])
    i += 1

  # Add title and labels

  plt.title(model)
  plt.xlabel(parameter)
  plt.ylabel('Score')

  plt.legend()

  plt.tight_layout()
  plt.show() 

In [53]:
scorecols = ['MAE']
plotscores(lgbdf,'max_depth',scorecols,'LGBMRegressor')

In [55]:
def metrics(ytest,pred,trainx):
  print("MAE", mae(ytest, pred))
  print("MSE", mse(ytest,pred))
  print("RMSE", np.sqrt(mse(ytest,pred)))
  r2score = r2(ytest,pred)
  print("r2score", r2score)
  n=len(trainx)
  k=4
  adj_r2_score = 1 - ((1-r2score)*(n-1)/(n-k-1))
  print("adjr2lis", adj_r2_score)

In [ ]:
# model = XGBRegressor(n_estimators=val,gamma=l,max_depth=m,objective='reg:squarederror')
# model.fit(xtrain, ytrain)
predtest = model.predict(X_test)
metrics(y_test,predtest,X_test)
# maescore = mae(y_test, predtest)

In [56]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0,test_size=0.2)
for val in [125,150,250,500,750,1000]:
  for l in [1,3,5,7,9]: 
    for m in [3,5,7,10]:
      model = XGBRegressor(n_estimators=val,gamma=l,max_depth=m,objective='reg:squarederror')
      model.fit(xtrain, ytrain)
      pred = model.predict(xtest)
      maescore = mae(ytest, pred)
      print("max_depth: " + str(m) + " and gamma : " + str(l) + " and n_estimators : " + str(val) + " and MAE : " + str(maescore))

In [57]:
## Define our candidate hyperparameters
#hp_candidates = [{'n_neighbors': [2,3,4,5,6], 'weights': ['uniform','distance']}]
xtrain, xtest, ytrain, ytest = train_test_split(minscaled, y, random_state=0,test_size=0.2)
for n in [2,3,4,5,6,7,8,9,10]:
  for w in ['uniform','distance']: 
    model = KNeighborsRegressor(n_neighbors=n,weights=w)
    model.fit(xtrain, ytrain)
    pred = model.predict(xtest)
    maescore = mae(ytest, pred)
    print("neighbors: " + str(n) + " and weights : " + w + " and MAE : " + str(maescore))

In [58]:
xtrain, xtest, ytrain, ytest = train_test_split(stdscaled, y, random_state=0,test_size=0.2)
for n in [2,3,4,5,6,7,8,9,10]:
  for w in ['uniform','distance']: 
    model = KNeighborsRegressor(n_neighbors=n,weights=w)
    model.fit(xtrain, ytrain)
    pred = model.predict(xtest)
    maescore = mae(ytest, pred)
    print("neighbors: " + str(n) + " and weights : " + w + " and MAE : " + str(maescore))

In [59]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0,test_size=0.2)
for n in [2,3,4,5,6,7,8,9,10]:
  for w in ['uniform','distance']: 
    model = KNeighborsRegressor(n_neighbors=n,weights=w)
    model.fit(xtrain, ytrain)
    pred = model.predict(xtest)
    maescore = mae(ytest, pred)
    print("neighbors: " + str(n) + " and weights : " + w + " and MAE : " + str(maescore))

In [60]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0,test_size=0.2)
for n in [5,10,15,20,25,50]:
  for w in ['uniform','distance']: 
    model = KNeighborsRegressor(n_neighbors=n,weights=w)
    model.fit(xtrain, ytrain)
    pred = model.predict(xtest)
    maescore = mae(ytest, pred)
    print("neighbors: " + str(n) + " and weights : " + w + " and MAE : " + str(maescore))

In [61]:
param_grid = {
    # 'bootstrap': [True],
    'max_depth': [1,2,3,4,5,6,7,8,9,10],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1.0],
    # 'min_samples_leaf': [3, 4, 5],
    # 'min_samples_split': [8, 10, 12],
    # 'n_estimators': [1000,1500]# 300, 1000
}
lgb = LGBMRegressor()
grid_search = GridSearchCV(estimator = lgb, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x,y)
print("Best parameters:", grid_search.best_params_)

In [62]:
#using the best parameters mentioned above for lgbm
lgb = LGBMRegressor(max_depth=10,learning_rate=0.1)
lgb.fit(xtrain, ytrain)
pred = lgb.predict(xtest)
maescore = mae(ytest, pred)
print("MAE : " + str(maescore))
# print("================================")

In [63]:
for val in [1,5,10,15,20,25,30,50,75,100]:
  for l in [0.0001, 0.001, 0.01, 0.1, 1.0]:
    lgb = LGBMRegressor(max_depth=val,learning_rate=l,num_leaves=2*val)
    lgb.fit(xtrain, ytrain)
    pred = lgb.predict(xtest)
    maescore = mae(ytest, pred)
    print("max_depth: " + str(val) + " and rate : " + str(l) + " and MAE : " + str(maescore))
    print("================================")

In [64]:
x.head()

In [65]:
extra = x.copy(deep=True)

In [66]:
extra['ATxRH'] = extra['AT'] * extra['RH']
extra['AT3'] = extra['AT']*extra['AT']*extra['AT']
extra['ATxAP2'] = extra['AT']*extra['AP']*extra['AP']
extra['VxRH'] = extra['V'] * extra['RH']
extra.head()

In [67]:
x2 = x.drop(['V','RH'],axis=1)
x2.head()

In [68]:
names = []
scores = []
maes = []

scorer = make_scorer(mae)
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True) 
    score = cross_val_score(model, x2, y, cv=kfold, scoring=scorer).mean()
    
    names.append(name)
    scores.append(score)

kf_cross_val = pd.DataFrame({'Name': names, 'Score': scores})
print(kf_cross_val)

In [69]:
corr=df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
fig, ax = plt.subplots(figsize=(8,8)) 
sns.heatmap(corr, mask=mask,  center=0, linewidths=1, annot=True, fmt=".3f", ax = ax)
plt.show()

In [70]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 0)
rf.fit(xtrain, ytrain)
rf.feature_importances_

In [71]:
plt.barh(xtrain.columns, rf.feature_importances_)

In [72]:
data1 = df.loc[:,['AT']]
data2 = df.loc[:,['AP','V']]
data3 = df.loc[:,['AT','V']]

In [73]:
data = [('AT',data1),('AP,V',data2),('AT,V',data3)]

In [74]:
# names = []
# scores = []
# maes = []
# datanames = []
for name, model in models:
  for dfname,df1 in data:
    xtrain, xtest, ytrain, ytest = train_test_split(df1, y, random_state=0,test_size=0.2)
    model.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    # scores.append(accuracy_score(ytest, y_pred))
    print("Model : " + name + ", Dataset : " + dfname + ", MAE : " + str(mae(ytest, y_pred)))
    # print(dfname)
    # print(mae(ytest, y_pred))
    # maes.append(mae(ytest, y_pred))
    # names.append(name)
    # datanames.append(dfname)

# tr_split = pd.DataFrame({'Name': names, 'Data':datanames, 'Score': scores, 'MAE': maes})
# print(tr_split)

In [75]:
xtrain, xtest, ytrain, ytest = train_test_split(data3, y, random_state=0,test_size=0.2)
for val in [5,10,20,50,100,200,250,500,750,1000,1250,1500,1750,2000]:
  rf = RandomForestRegressor(n_estimators = val, random_state = 0)
  rf.fit(xtrain, ytrain)
  pred = rf.predict(xtest)
  maescore = mae(ytest, pred)
  print("n_estimators: " + str(val) + " and MAE : " + str(maescore))
  print("================================")

In [76]:
df['ATxV'] = df['AT'] * df['V']
df['ATxAP'] = df['AT'] * df['AP']
df['ATxRH'] = df['AT'] * df['RH']

In [77]:
df.head()

In [78]:
dfx = df.drop(['ID','EP'],axis=1)

In [79]:
xtrain, xtest, ytrain, ytest = train_test_split(dfx, y, random_state=0,test_size=0.2)

In [80]:
rf = RandomForestRegressor(random_state = 0)
rf.fit(xtrain, ytrain)
rf.feature_importances_

In [81]:
plt.barh(xtrain.columns, rf.feature_importances_)

In [82]:
pred = rf.predict(xtest)
mae(ytest, pred)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(dfx, y, random_state=0,test_size=0.2)
for val in [5,10,20,50,100,200,250,500,750,1000,1250,1500,1750,2000]:
  rf = RandomForestRegressor(n_estimators = val, random_state = 0)
  rf.fit(xtrain, ytrain)
  pred = rf.predict(xtest)
  maescore = mae(ytest, pred)
  print("n_estimators: " + str(val) + " and MAE : " + str(maescore))
  print("================================")

In [85]:
dfx2 = df.loc[:,['ATxV']]
xtrain, xtest, ytrain, ytest = train_test_split(dfx2, y, random_state=0,test_size=0.2)
rf = RandomForestRegressor(random_state = 0)
rf.fit(xtrain, ytrain)
pred = rf.predict(xtest)
mae(ytest, pred)

In [86]:
dfx3 = df.loc[:,['ATxV','AP','RH']]
xtrain, xtest, ytrain, ytest = train_test_split(dfx3, y, random_state=0,test_size=0.2)
rf = RandomForestRegressor(random_state = 0)
rf.fit(xtrain, ytrain)
pred = rf.predict(xtest)
mae(ytest, pred)

In [87]:
test = pd.read_csv('/kaggle/input/lab-02-introduction-to-machine-learning/test.csv')
test.head()

In [88]:
test.shape

In [89]:
testx = test.loc[:,['AT','V','AP','RH']]
testx.head()

In [90]:
scaler = MinMaxScaler()
# transform data
# minscaled = x.copy(deep=True)
testx[testx.columns] = scaler.fit_transform(testx[testx.columns])
# stdscaled = scaler.fit_transform(x)
testx.head()

In [93]:
model = XGBRegressor(n_estimators=750,gamma=1,max_depth=10,objective='reg:squarederror') #testpred12
model.fit( x, y)
pred = model.predict(testx) 
test['EP'] = pred
test.head()

In [94]:
lgb = LGBMRegressor(max_depth=100,learning_rate=0.1,num_leaves=200) #testpred4
lgb.fit(minscaled, y)
pred = lgb.predict(testx)
test['EP'] = pred
test.head()

In [95]:
model = XGBRegressor(n_estimators=1000,gamma=1,max_depth=7,objective='reg:squarederror') #testpred5
model.fit(x, y)
pred = model.predict(testx)
test['EP'] = pred
test.head()

In [96]:
model = KNeighborsRegressor(n_neighbors=4,weights='distance') #testpred6
model.fit(minscaled, y)
pred = model.predict(testx)
test['EP'] = pred
test.head()

In [97]:
model = KNeighborsRegressor(n_neighbors=7,weights='distance') #testpred8
model.fit(x, y)
pred = model.predict(testx)
test['EP'] = pred
test.head()

In [98]:
model = XGBRegressor(n_estimators=1000,gamma=1,max_depth=7,objective='reg:squarederror') #testpred9
model.fit(x, y)
pred = model.predict(testx)
test['EP'] = pred
test.head()

In [99]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 0)
rf.fit(x, y)
pred = rf.predict(testx)
test['EP'] = pred
test.head()

In [100]:
lgb = LGBMRegressor(max_depth=75,learning_rate=0.1,num_leaves=2*75) #testpred3
lgb.fit(x, y)
pred = lgb.predict(testx)
test['EP'] = pred
test.head()

In [101]:
# test.to_csv('testpred11.csv',index=False) 
# files.download('testpred11.csv')

In [102]:
test.shape

In [103]:
test['ATxV'] = test['AT'] * test['V']
test['ATxAP'] = test['AT'] * test['AP']
test['ATxRH'] = test['AT'] * test['RH']

In [104]:
dfx.head()

In [105]:
testx = test.drop(['ID'],axis=1)
testx.head()